In [0]:
%tensorflow_version 1.x

import numpy as np
import keras
from keras.datasets import mnist
import sys
from scipy.stats import entropy
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, SpatialDropout2D
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.regularizers import l2
from keras import backend as K
from sklearn import metrics

from google.colab import drive
drive.mount("/content/gdrive") 

TensorFlow 1.x selected.


Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
def predict_with_uncertainty(f, x, n_iter=100):
    """Function generating non-deterministic predictions using MC dropout and returning the mean of these predictions
    Adapted from: https://stackoverflow.com/questions/43529931/how-to-calculate-prediction-uncertainty-using-keras
    #Arguments:
        f: function mapping model input and Keras backend learning_phase flag to model output
        x: input
        n_iter: number of repreated MC dropout predictions per point
    #Returns:
        Mean of MC dropout predictions
    """
    result = np.zeros((n_iter,x.shape[0], 2))
    for i in range(n_iter):
        predictions = np.array(f((x, 1))[0])
        result[i,:, :] = predictions
    prediction = result.mean(axis=0)
    return prediction

def predict_pool_with_uncertainty(f, x, n_iter=50):
    """Function generating and returning non-deterministic predictions using MC dropout
    Adapted from: https://stackoverflow.com/questions/43529931/how-to-calculate-prediction-uncertainty-using-keras
    #Arguments:
        f: function mapping model input and Keras backend learning_phase flag to model output
        x: input
        n_iter: number of repreated MC dropout predictions per point
    #Returns:
        All MC dropout predictions
    """
    result = np.zeros((n_iter,x.shape[0], 2))
    for i in range(n_iter):
        predictions = np.array(f((x, 1))[0])
        result[i,:, :] = predictions
    return result


def run_model (train_data_indices):
  """Trains a Keras model with the training points specified by train_data_indices and evaluates model on test data
  #Arguments:
      train_data_indices: indices of current training points within X_train_All
  #Returns:
      Test accuracy and Keras model
  """
  X_train = np.expand_dims(X_train_All[train_data_indices], axis=1)
  y_train = y_train_All[train_data_indices]
  y_train[y_train==7] = 1
  y_train[y_train==9] = 0
  y_train = keras.utils.to_categorical(y_train, num_classes=2)
  train_size = y_train.shape[0]
  Weight_Decay = 0.0005/train_size
  dropout_prob = 0.30
  batch_size=128
  nb_filters = 35
  nb_pool = 3
  nb_conv = 4
  img_rows = img_cols = 28
  nb_classes = 2
  model = Sequential()
  model.add(Convolution2D(nb_filters, nb_conv, strides=1, data_format="channels_first", input_shape=(1, img_rows, img_cols)))
  model.add(Activation('relu'))
  model.add(Convolution2D(nb_filters, nb_conv, data_format="channels_first", strides=2))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool), data_format="channels_first"))
  model.add(Dropout(dropout_prob))
  model.add(Flatten())
  model.add(Dense(128, W_regularizer=l2(Weight_Decay)))
  model.add(Activation('relu'))
  model.add(Dropout(dropout_prob))
  model.add(Dense(nb_classes, W_regularizer=l2(Weight_Decay)))
  model.add(Activation('softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam')
  model.fit(X_train, y_train, epochs=300, batch_size=128, verbose=0)
  f = K.function([model.layers[0].input, K.learning_phase()],[model.layers[-1].output])
  y_test_output = predict_with_uncertainty(f, X_test, n_iter=100)
  y_test_predictions = np.argmax(y_test_output, axis=1)
  return [np.sum(y_test_predictions==y_test_original)/(y_test_original.shape[0]), model]



In [0]:

def one_center_exact (labeled_embeddings, embedding_arr, all_pairwise_distances, outlier_proportion):
  """Exact implementation of robust k-Center with batch size 1 by evaluating the cost with each candidate point selected as a center
  #Arguments:
    labeled_embeddings: embeddings corresponding to training data (shape: [size_train, embedding_dim])
    embedding_arr: embeddings corresponding to unlabeled candidate points (shape: [num_candidates, embedding_dim])
    all_pairwise_distances: array containing all pairwise distances between embeddings (for both labeled & unlabeled points)
    outlier_proportion: proportion of unlabeled points that is set to equal the outlier tolerance for robust k-center algorithm
  #Returns:
    robust k-Center cost and indices of queried points
  """
  num_unlabeled_embeddings = len(embedding_arr)
  num_labeled_embeddings = len(labeled_embeddings)
  outlier_tolerance = int(outlier_proportion * num_unlabeled_embeddings)
  all_set_distances = np.zeros((num_unlabeled_embeddings, num_unlabeled_embeddings))
  all_costs = np.zeros(num_unlabeled_embeddings)
  #Creating distance matrix all_set_distances: minimum distance from each unlabeled point to the set of labeled points 
  #when each unlabeled point is added to the set of labeled points. First axis: newly added point, second axis: unlabeled point.
  for queried_ind in range(num_unlabeled_embeddings):
    all_distances = np.concatenate((all_pairwise_distances[0:num_labeled_embeddings, num_labeled_embeddings:],np.expand_dims(all_pairwise_distances[num_labeled_embeddings+queried_ind, num_labeled_embeddings:], axis=0)))
    all_set_distances[queried_ind, :] = np.amin(all_distances, axis=0)
    cost_determining_ind = np.flip(np.argsort(all_set_distances[queried_ind, :]))[outlier_tolerance]
    all_costs[queried_ind] = all_set_distances[queried_ind, cost_determining_ind]
  cost = np.amin(all_costs)
  acq_ind = np.argmin(all_costs)
  return [cost, acq_ind]



In [0]:
#Active learning parameters/settings

#Proportion of unlabeled candidate points taken to be outlier tolerance for robust k-Center
outlier_proportion = 0.001

dropout_prob = 0.3
num_experiments = 3
num_acquisitions = 200
pool_sample_size = 2500
num_masks = 50
embedding_dim = 128
#binary_search_itr = 5
#Xi = pool_sample_size//1000
batch_size = 1

In [0]:
#Loading data
data_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/MNIST/"
starting_ind_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/MNIST/Binary_7_9_AL_Scripts/"
results_path = "/content/gdrive/My Drive/FINAL_PAPER_ACTIVE_LEARNING_EXP/MNIST/Binary_7_9_AL_Results/"
train_data = np.loadtxt(data_path + "mnist_train.csv", 
			delimiter=",")
test_data = np.loadtxt(data_path + "mnist_test.csv", 
			delimiter=",") 
y_train_All = train_data[:,0]
y_test = test_data[:,0]
X_train_All = train_data[:,1:].reshape((60000,28,28))
X_test = test_data[:,1:].reshape((10000,28,28)) 
train_ind = np.concatenate((np.argwhere(y_train_All==7), np.argwhere(y_train_All==9))).flatten()
test_ind = np.concatenate((np.argwhere(y_test==7), np.argwhere(y_test==9))).flatten()
y_test = y_test[test_ind]
y_test[y_test==9] = 0
y_test[y_test==7]= 1
y_test_original = y_test
y_test = keras.utils.to_categorical(y_test, num_classes=2)
X_test = np.expand_dims(X_test[test_ind], axis=1)

#Iterating across active learning experiments, each of which starts with a different initial training set

for e in range(2,3):
  acc_file = "CoreSetExact_BS" + str(batch_size)+"_Acc_Ind" + str(e+1) + ".npy"
  ind_file = "CoreSetExact_BS"+str(batch_size)+"_Ind_Ind" + str(e+1) + ".npy"
  exp_acc = []
  #exp_acc = list(np.load(results_path+acc_file))
  train_data_indices = list(np.load(starting_ind_path + 'trainindices' + str(e+1) + '.npy'))
  #train_data_indices = list(np.load(results_path+ind_file))
  pool_indices = [i for i in train_ind if i not in train_data_indices]
  #num_acquisitions = num_acquisitions - batch_size * (len(exp_acc) - 1)

  #Looping over active learning iterations
  for acq in range(num_acquisitions//batch_size + 1):
    #Training and evaluating model
    pool_ind_sample = np.random.choice(pool_indices, pool_sample_size, replace=False)
    model_results = run_model(train_data_indices)
    exp_acc.append(model_results[0])
    if acq != num_acquisitions//batch_size:
      curr_size_train = len(train_data_indices)
      model = model_results[1]
      #Obtaining embeddings for robust k-Center
      X_k_center = np.expand_dims(np.concatenate((X_train_All[train_data_indices], X_train_All[pool_ind_sample])),axis=1)
      f_embed = K.function([model.layers[0].input, K.learning_phase()],
                    [model.layers[-4].output])
      all_embeddings = np.array(f_embed((X_k_center, 0))).reshape((X_k_center.shape[0], embedding_dim))
      pairwise_distances = metrics.pairwise_distances(all_embeddings)
      print(pairwise_distances.shape)
      #robust k-Center acquisition
      [cost, acq_ind_ind] = one_center_exact (all_embeddings[0:curr_size_train, :], all_embeddings[curr_size_train:, :], pairwise_distances, outlier_proportion)
      acq_ind = pool_ind_sample[acq_ind_ind]
      train_data_indices.append(acq_ind)
      pool_indices.remove(acq_ind)
    #Saving results
    np.save(results_path+acc_file, np.array(exp_acc))
    np.save(results_path+ind_file, np.array(train_data_indices))
    print('Exp: ' + str(e+1) + ', train size: ' + str(len(train_data_indices)) + ', Number elapsed iterations: ' + str(len(exp_acc)) + ", last acc: " + str(exp_acc[-1]))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_regularizer=<keras.reg...)`



(2510, 2510)
Exp: 3, train size: 11, Number elapsed iterations: 1, last acc: 0.819833087874325


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_regularizer=<keras.reg...)`


(2511, 2511)
Exp: 3, train size: 12, Number elapsed iterations: 2, last acc: 0.8085419734904271


...

(2708, 2708)
Exp: 3, train size: 209, Number elapsed iterations: 199, last acc: 0.9823269513991163


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_regularizer=<keras.reg...)`


(2709, 2709)
Exp: 3, train size: 210, Number elapsed iterations: 200, last acc: 0.9788905252822778


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:42: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(2, kernel_regularizer=<keras.reg...)`


Exp: 3, train size: 210, Number elapsed iterations: 201, last acc: 0.9926362297496318
